In [51]:

import os
import sys
from dotenv import load_dotenv
from backend.src.rag_pipeline import initialize_rag_pipeline

In [50]:
sys.path.append(os.path.abspath(os.path.join('..')))
load_dotenv(os.path.abspath(os.path.join('..', '.env')))

True

In [52]:
# Check API Key
api_key = os.getenv("API_KEY")
print(f"API Key: {'Found' if api_key else 'Not found'}")
if api_key:
    print(f"Key starts with: {api_key[:8]}...")


API Key: Found
Key starts with: gsk_oYYq...


In [53]:
# Test Groq API
from groq import Groq

client = Groq(api_key=api_key)
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": "Hello"}],
    temperature=0.2,
    max_tokens=10
)
print(f"API Test Response: {response.choices[0].message.content}")


API Test Response: Hello. How can I assist you today?


In [54]:
# Initialize RAG Pipeline
faiss_store_path = os.path.abspath(os.path.join('..', 'faiss_store'))
rag_pipeline = initialize_rag_pipeline(storage_path=faiss_store_path)

print(f"Vector Database Size: {rag_pipeline.vector_db.size} chunks")


Initializing RAG Pipeline...
TextChunker initialized (size=1500, overlap=300)
Loading embedding model: thenlper/gte-large...
Model loaded successfully (batch_size=32)
VectorDatabase initialized at: c:\Users\nirma\OneDrive - Lambton College\Documents\1.ML_AL_PROJECTS\Ongoing projects\Crop-Advisory-Assistant\faiss_store
Loaded database from c:\Users\nirma\OneDrive - Lambton College\Documents\1.ML_AL_PROJECTS\Ongoing projects\Crop-Advisory-Assistant\faiss_store (1701 vectors)
Connecting to AI model...
RAGPipeline initialized (Chunker → Embedder → VectorDB → LLM)
RAG system ready - no documents loaded. Use API endpoints to upload documents.
RAG system ready!
Vector Database Size: 1701 chunks


In [55]:
# Test Question Answering
query = "What is the best way to manage pests in a wheat crop?"
result = rag_pipeline.answer(query)

print(f"Question: {query}")
print(f"Answer: {result['answer']}")
print(f"Sources: {len(result['sources'])}")
for i, source in enumerate(result['sources'], 1):
    print(f"  {i}. {source['source']}")


Retrieving documents for: 'What is the best way to manage pests in a wheat crop?'
Retrieved 4 documents
Question: What is the best way to manage pests in a wheat crop?
Answer: The best way to manage pests in a wheat crop is through Integrated Pest Management (IPM) principles. IPM involves a holistic approach to managing pests, focusing on preventing infestations and using a combination of techniques to control pest populations. 

Some key IPM principles for managing pests in a wheat crop include:

1. **Crop rotation and intercropping**: Rotate wheat with other crops to break the life cycle of pests and reduce the risk of infestation. Intercropping with other crops can also help to minimize pest pressure.
2. **Adequate cultivation techniques**: Use seedbed sanitation, timely sowing, optimum plant population, conservation tillage, thinning, and direct sowing to reduce pest infestations.
3. **Use of pest-resistant cultivars**: Choose wheat varieties that are resistant or tolerant to commo

In [56]:
# Test Document Retrieval
query = "pest management"
retrieved_docs = rag_pipeline.retrieve(query, top_k=3)

print(f"Query: {query}")
print(f"Retrieved {len(retrieved_docs)} documents:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"  {i}. {doc['metadata']['source']}")
    print(f"     Distance: {doc['distance']:.4f}")
    print(f"     Text: {doc['metadata']['text'][:100]}...")


Retrieving documents for: 'pest management'
Retrieved 3 documents
Query: pest management
Retrieved 3 documents:
  1. C:\Users\nirma\OneDrive - Lambton College\Documents\1.ML_AL_PROJECTS\Ongoing projects\Crop-Advisory-Assistant\backend\api\routes\..\..\..\data\raw\Compilation_techniques_organic_agriculture_rev.pdf
     Distance: 0.2175
     Text: 61 
 
 8. PEST AND DISEASE MANAGEMENT  
IN ORGANIC AGRICULTURE  
Pest and disease management consist...
  2. C:\Users\nirma\OneDrive - Lambton College\Documents\1.ML_AL_PROJECTS\Ongoing projects\Crop-Advisory-Assistant\backend\api\routes\..\..\..\data\raw\Rice_Manuals_i7107en.pdf
     Distance: 0.2227
     Text: silver shoot  
 
Management Practices  
 Light Trapping  
 Visit the field regularly and r ecommen...
  3. C:\Users\nirma\OneDrive - Lambton College\Documents\1.ML_AL_PROJECTS\Ongoing projects\Crop-Advisory-Assistant\backend\api\routes\..\..\..\data\raw\Agriculture_practices_Guidelines_volume_1_2.pdf
     Distance: 0.2326
     Text: m